# Elastic job demo

SETUP and EXECUTE Elastic Jobs on ElasticJobDb

we will start by creating a master key in our database. If the key has already been created it will get an error, which is not a big deal.

In [2]:
create master key;

Next we will add scoped credentials for the user databases and master database on the server

In [3]:
if not exists(select 1 from sys.database_scoped_credentials where name = 'elasticjob')
  create database scoped credential elasticjob with identity = 'elasticjob', secret = 'PassMn2020!';
GO

if not exists(select 1 from sys.database_scoped_credentials where name = 'masterjob')
  create database scoped credential masterjob with identity = 'masterjob', secret = 'PassMn2020!';
GO

Following code cleans up jobs created in previous demos

In [4]:
exec jobs.sp_delete_job @job_name = 'AddNewTableOrRowRightNow', @force=1;
exec jobs.sp_delete_target_group @target_group_name = 'tgWideWorldDWs';
exec jobs.sp_delete_job @job_name = 'AddGuidTable', @force=1;
exec jobs.sp_delete_target_group @target_group_name = 'tgAllDbs';

The next two code steps will create a new target group and add a single database to that group.

In [5]:
exec jobs.sp_add_target_group 'tgWideWorldDWs';

In [6]:
exec jobs.sp_add_target_group_member 
	@target_group_name = N'tgWideWorldDWs'
	,@target_type = N'SqlDatabase'
	,@server_name = N'<<servername>>.database.windows.net'
	,@database_name = N'WideWorldDW_1'
;

Let's check the jobs tables for the group and group membership we just created

In [7]:
select * from jobs.target_groups;
select * from jobs.target_group_members where target_group_name = N'tgWideWorldDWs'

Let's add our first job. Reminder that a job is a placeholder for steps which we will add next.

In [8]:
exec jobs.sp_add_job @job_name = 'AddNewTableOrRowRightNow'

Now that we have our job, we will add a step. This step will create a new table called new table in our target database.

In [9]:
-- add a job step
exec jobs.sp_add_jobstep 
	@job_name = 'AddNewTableOrRowRightNow'
	, @command = 'if not exists (select 1 from sys.tables where object_id = object_id(''NewTable''))
					create table dbo.NewTable (col1 uniqueidentifier null,col2 datetime2(7) null);
                    insert into dbo.NewTable values (NewID(), getdate()); '
	, @credential_name = 'elasticjob'
	, @target_group_name = 'tgWideWorldDWs'

We will use the following stored procedure to execute the job manually

In [10]:
-- manually execute the job
exec jobs.sp_start_job 'AddNewTableOrRowRightNow'

This next procedure will show you the job execution and current status

In [11]:
-- check status of job executions
select * from jobs.job_executions;

Let's check the server listing and browse to the database to see the new table that was created.

in our next step we will create a target group for a server

In [12]:
exec jobs.sp_add_target_group 'tgAllDbs';

In this step we will add the server to the target group that we created as the only member

In [13]:
exec jobs.sp_add_target_group_member 
	@target_group_name = N'tgAllDbs'
	,@target_type = N'SqlServer'
	,@refresh_credential_name = N'masterjob' -- credential needed to view the current list of Dbs
	,@server_name = N'<<servername>>.database.windows.net'
;

Let's check out the job target groups to see what we added. Notice that the target type is SQL Server not a database.

In [14]:
select * from jobs.target_group_members where target_group_name = N'tgAllDbs'

In this step we will add a job for next table

In [15]:
exec jobs.sp_add_job @job_name = 'AddGuidTable'

Now we add a job step to create that table in all the databases on the server

In [16]:
-- add a job step
exec jobs.sp_add_jobstep 
	@job_name = 'AddGuidTable'
	, @command = 'if not exists (select 1 from sys.tables where object_id = object_id(''GuidTable''))
					create table dbo.GuidTable (col1 uniqueidentifier null, col2 datetime2(7) null); 
                    insert into dbo.GuidTable values (NewID(), getdate()); '
	, @credential_name = 'elasticjob'
	, @target_group_name = 'tgAllDbs'

Let's run the job manually

In [17]:
-- manually execute the job
exec jobs.sp_start_job 'AddGuidTable'

Time to check our job status

In [19]:
-- check status of job executions
select * from jobs.job_executions where job_name = 'AddGuidTable';

In the results above, you will see there are a couple failures. Those failures are due to the fact that not all of the databases are accessible to the identity we are using to execute the job. As a result the job will actually stay active while it waits for the substeps to continue or finish.

This next query will return the active jobs With start time.

In [20]:
select * from jobs.job_executions where is_active = 1 and job_name = 'AddGuidTable'

In [ ]:
 order by start_time desc;

Because we don't have the proper credentials, we need to cancel the job. We'll need to pull the active job execution ID from the query above and run the stop job stored procedure.

In [21]:
-- because we don't have credentials set for one database we need to cancel. Use the following with the active job execution guid from the previous query
DECLARE @job as varchar(100);
select @job = job_execution_id from jobs.job_executions where is_active = 1 and job_name = 'AddGuidTable';
SELECT @job;
exec jobs.sp_stop_job @job;

The next query will actually exclude a database from our list on the server.

In [22]:
-- check results -- need to run query on target databases

-- exclude the problem databases
exec jobs.sp_add_target_group_member
	@target_group_name = 'tgAllDbs'
	, @membership_type = 'Exclude'
	, @target_type = 'SQLDatabase'
	, @server_name = 'dataonwheels-eus-02.database.windows.net'
	, @database_name = 'WideWorldImportersDW-Standard'
;

the job database is on the same server. The principal does not have permissions here either

In [23]:
exec jobs.sp_add_target_group_member
	@target_group_name = 'tgAllDbs'
	, @membership_type = 'Exclude'
	, @target_type = 'SQLDatabase'
	, @server_name = 'dataonwheels-eus-02.database.windows.net'
	, @database_name = 'ElasticJobDb'
;

Let's re execute the job and check the status

In [24]:
exec jobs.sp_start_job 'AddGuidTable'

In [26]:
-- check status of job executions
select * from jobs.job_executions where is_active = 1 and job_name = 'AddGuidTable' order by start_time desc;

As you can see in those results and by checking the server browser, we have added the tables to all of the targeted databases on the server. This concludes the demo for last jobs.